<a href="https://colab.research.google.com/github/Phantom1502/Deep-Learning/blob/main/Character_level_LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

#Config drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

import os
os.chdir('/content/gdrive/My Drive/')

from sklearn.model_selection import train_test_split
import pandas as pd

Mounted at /content/gdrive/


# PREPARING THE DATA

Let’s start by downloading the corpus and converting it to lowercase.

In [13]:
import tensorflow as tf
import keras
import numpy as np
path = tf.keras.utils.get_file('T%C3%94I_TH%E1%BA%A4Y_HOA_V%C3%80NG_TR%C3%8AN_C%E1%BB%8E_XANH.txt', origin='http://45.77.183.94/documents/2/T%C3%94I_TH%E1%BA%A4Y_HOA_V%C3%80NG_TR%C3%8AN_C%E1%BB%8E_XANH.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

352256/343815 [==============================] - 0s 0us/step
Corpus length: 258669


In [14]:
text[0:500]

'chú đàn bảo tôi:\n\n- con xòe tay ra cho chú xem nào!\n\ntôi co những ngón tay lại, nắm thật chặt và giấu ra sau lưng:\n\n- tay con sạch cơ mà. hồi sáng con đã rửa tay rồi.\n\nchú đàn phì cười:\n\n- chú có định khám tay con đâu. con xòe tay ra để chú xem con có mấy cái hoa tay thôi.\n\nđằng sau lưng, hai bàn tay tôi lỏng đi. tôi chìa bàn tay trái ra trước mặt chú đàn, thắc mắc:\n\n- hoa tay là gì hở chú?\n\nchú đàn dựng mắt nhìn tôi:\n\n- con lớn từng này rồi mà không biết hoa tay là gì à?\n\nchú cầm lấy bàn tay tô'

Next, you’ll extract partially overlapping sequences of length maxlen, one-hot encode
them, and pack them in a 3D Numpy array x of shape (sequences, maxlen,
unique_characters). Simultaneously, you’ll prepare an array y containing the corresponding targets: the one-hot-encoded characters that come after each extracted
sequence.

In [26]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
# doan code tren lay chuoi 60 ky tu, va ky tu tiep theo cua chuoi nay de lam tap train, label

print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 86203
Unique characters: 104
{'\n': 0, ' ': 1, '!': 2, '"': 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, ':': 12, '?': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'g': 19, 'h': 20, 'i': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'x': 34, 'y': 35, 'à': 36, 'á': 37, 'â': 38, 'ã': 39, 'è': 40, 'é': 41, 'ê': 42, 'ì': 43, 'í': 44, 'ò': 45, 'ó': 46, 'ô': 47, 'õ': 48, 'ù': 49, 'ú': 50, 'ý': 51, 'ă': 52, 'đ': 53, 'ĩ': 54, 'ũ': 55, 'ơ': 56, 'ư': 57, 'ạ': 58, 'ả': 59, 'ấ': 60, 'ầ': 61, 'ẩ': 62, 'ẫ': 63, 'ậ': 64, 'ắ': 65, 'ằ': 66, 'ẳ': 67, 'ẵ': 68, 'ặ': 69, 'ẹ': 70, 'ẻ': 71, 'ẽ': 72, 'ế': 73, 'ề': 74, 'ể': 75, 'ễ': 76, 'ệ': 77, 'ỉ': 78, 'ị': 79, 'ọ': 80, 'ỏ': 81, 'ố': 82, 'ồ': 83, 'ổ': 84, 'ỗ': 85, 'ộ': 86, 'ớ': 87, 'ờ': 88, 'ở': 89, 'ỡ': 90, 'ợ': 91, 'ụ': 92, 'ủ': 93, 'ứ': 94, 'ừ': 95, 'ử': 96, 'ữ': 97, 'ự': 98, 'ỳ': 99, 'ỵ': 100, 'ỷ': 101, 'ỹ': 102, '–': 103}
Vectorization...


# BUILDING THE NETWORK
This network is a single LSTM layer followed by a Dense classifier and softmax over all
possible characters. But note that recurrent neural networks aren’t the only way to do
sequence data generation; 1D convnets also have proven extremely successful at this
task in recent times.

In [27]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Because your targets are one-hot encoded, you’ll use categorical_crossentropy as
the loss to train the model.

In [30]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
model.fit(x, y, batch_size=128, epochs=60)

Epoch 1/60
674/674 [==============================] - 144s 213ms/step - loss: 0.9522
Epoch 2/60
674/674 [==============================] - 142s 211ms/step - loss: 0.9500
Epoch 3/60
674/674 [==============================] - 142s 211ms/step - loss: 0.9453
Epoch 4/60
674/674 [==============================] - 141s 210ms/step - loss: 0.9436
Epoch 5/60
674/674 [==============================] - 143s 213ms/step - loss: 0.9402
Epoch 6/60
674/674 [==============================] - 150s 222ms/step - loss: 0.9342
Epoch 7/60
674/674 [==============================] - 150s 222ms/step - loss: 0.9348
Epoch 8/60
674/674 [==============================] - 146s 217ms/step - loss: 0.9314
Epoch 9/60
674/674 [==============================] - 146s 216ms/step - loss: 0.9279
Epoch 10/60
674/674 [==============================] - 147s 218ms/step - loss: 0.9223
Epoch 11/60
674/674 [==============================] - 145s 215ms/step - loss: 0.9173
Epoch 12/60
674/674 [==============================] - 145s 215

In [32]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
674/674 [==============================] - 137s 198ms/step - loss: 2.0431
--- Generating with seed: "ớn:

- a, thằng tường đứng lên được rồi, mẹ ơi!

ngày hôm đó"
------ temperature: 0.2
ớn:

- a, thằng tường đứng lên được rồi, mẹ ơi!

ngày hôm đó cho thấy thấy thấy đánh nghe tôi thấy người chúc thấy nghe thấy nghe tôi thấy thấy thấy chú đàn thấy này chúc chúc cho tiếng thấy thấy nghe tôi thấy chú đàn chuyện con chi thấy thấy thấy nghe tôi chuyện trong thấy thấy thấy chú đàn và thấy thấy thấy nghe tôi chuyện nghe tôi chúa chuyện trong nghe tôi thấy thấy nhà chú đàn thấy được thầy thấy chúa cho con chi chuyện thấy thấy đang nghe thấy theo t------ temperature: 0.5
thấy chúa cho con chi chuyện thấy thấy đang nghe thấy theo thấy trò tay là là tường giếng xa nghe tôi đái chi mà tôi tôi chúa tôi tích tôi tới thấy ngời ngững nhà tôi tôi được bời là đong thì hai nó đang tiếng tay và thắc rng lác thấy trên trác rống nữa tiư tao cho là tôi nghe tôi là khiến nhà mà thấy này là hoàng cho c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


không đch ở giả lắm vue động táu đứng giống như ai ba cảm ngày là vì cố thấy trùng miệng rất nữa con đất?


mội tiếng bắt nhưng thanh thì nựng chiếng bóc không khi tôi thống smo từ buồn tường, thằng tường đâu đó là khác thấy trước lo lãng là trang con mận, thấy nó rồi ra chú như vải lyên bất lên lhe.

- mày đứng lên tay nhà, sau một------ temperature: 1.2
ú như vải lyên bất lên lhe.

- mày đứng lên tay nhà, sau một dưng:

- em bật tặt vcong, tôi lổn ra chạy nhan c


như vậy, nắc lvư đã đất lấy tôi đột mọng ch:

- anh con chuồn củang mững háo:

- tải chìa biếti hành chhức mô cắn vhơn theo túi quanh, mây nay nhỏ ơi! cếu hả sọng nguyếng mông tìm phoót sơ. cuối cùng chuyện vnờ chu nữa xíu nó lại chuỗi trỏi tường đất láng.

những hiến ròn lút ngón trêu đó: tại trai giác rồi.

- cho cảm chuyện gì tôi. lớn cạnh tụepoch 12
674/674 [==============================] - 139s 207ms/step - loss: 1.2831
--- Generating with seed: "ại được toàn thể triều thần tôn lên giữ chức tể tướng.

tôi "
------ tem